In [ ]:
# added a parse_dates line for events df when importing so we don't have to do that separately:


In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
from tqdm import tqdm, trange

Here we import the data from the appropiate folder. This is different for the two members of the team so we wrapped it in a try block to account for this difference.

In [22]:
try:
    app_events = pd.read_csv('../talkingdata_DATA/app_events.csv')
    brand_df = pd.read_csv('../talkingdata_DATA/phone_brand_device_model.csv')
    label_categories = pd.read_csv('../talkingdata_DATA/label_categories.csv')
    app_labels = pd.read_csv('../talkingdata_DATA/app_labels.csv')
    events = pd.read_csv('../talkingdata_DATA/events.csv', parse_dates=['timestamp'])
    gender_age_train = pd.read_csv('../talkingdata_DATA/gender_age_train.csv')
    gender_age_test = pd.read_csv('../talkingdata_DATA/gender_age_test.csv')
except FileNotFoundError:
    app_events = pd.read_csv('data/app_events.csv', nrows = 1000000)
    brand_df = pd.read_csv('data/phone_brand_device_model.csv')
    label_categories = pd.read_csv('data/label_categories.csv')
    app_labels = pd.read_csv('data/app_labels.csv')
    events = pd.read_csv('data/events.csv', parse_dates=['timestamp'], nrows = 1000000)
    gender_age_train = pd.read_csv('data/gender_age_train.csv')
    gender_age_test = pd.read_csv('data/gender_age_test.csv')

In [27]:
#brand_df = pd.read_csv('../talkingdata_DATA/phone_brand_device_model.csv')

To get a feeling of how the data is structured we show the first rows for each dataframe

In [4]:
display(gender_age_train.head()) # want to predict group, involves multiclass classification

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31


In [5]:
display(app_events.head()) # event_id corresponds to that in events dataframe

,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1
1,2,-5720078949152207372,1,0
2,2,-1633887856876571208,1,0
3,2,-653184325010919369,1,1
4,2,8693964245073640147,1,1


In [6]:
display(events.head())
display(brand_df.head())

,event_id,device_id,timestamp,longitude,latitude
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
3,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
4,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66


,device_id,phone_brand,device_model
0,-8890648629457979026,小米,红米
1,1277779817574759137,小米,MI 2
2,5137427614288105724,三星,Galaxy S4
3,3669464369358936369,SUGAR,时尚手机
4,-5019277647504317457,三星,Galaxy Note 2


### Clean phone brand dataframe (brand_df)

In [9]:
display(brand_df.head()) # dataframe pre-cleanup

,device_id,phone_brand,device_model
0,-8890648629457979026,小米,红米
1,1277779817574759137,小米,MI 2
2,5137427614288105724,三星,Galaxy S4
3,3669464369358936369,SUGAR,时尚手机
4,-5019277647504317457,三星,Galaxy Note 2


As part of the cleanup we drop possible duplicate entries and (ALEX: dont fully understand whats happening here with the reset_index)

KATHERINE's explanation (can remove once understood)
    1. drop the duplicates in phone brand column to get only unique values, but then want consecutive values associated with each brand, so reset index with drop=True just resets the index and doesn't keep the old one, then the second reset_index() call changes that new consecutive index into a column (easier to use a column than the index itself sometimes)
    2. Set the index which we turned into a column to be titled phone_brand_id, giving us a df with consecutive values associated with each brand (could be a simpler way to do this with set or list, but this worked fine)

In [28]:
# dictionary of brand categories and their associated numbers/indices
brand_categories = brand_df[['phone_brand']].drop_duplicates().reset_index(drop=True).reset_index()
brand_categories = brand_categories.rename(columns = {'index': 'phone_brand_id'})

In [29]:
device_categories = brand_df[['device_model']].drop_duplicates().reset_index(drop=True).reset_index()
device_categories = device_categories.rename(columns = {'index':'device_model_id'})
# want a number for each phone-brand, not chinese name

In [30]:
# want a number for each phone-brand, not chinese name
brand_df = brand_df.merge(brand_categories, on='phone_brand')
brand_df = brand_df.merge(device_categories, on='device_model')
brand_df.drop(['device_model','phone_brand'], inplace=True, axis=1)
#brand_df.sort_values('device_id', inplace=True)

In [31]:
display(brand_df.head())

,device_id,phone_brand_id,device_model_id
0,-8890648629457979026,0,0
1,3933889243973057197,0,0
2,3379712123596745643,0,0
3,-5817422246078672037,0,0
4,-5926490704406813688,0,0


In [33]:
# Look at breakdown of top 5 phone brands: 'Proportion' represents the proportion of total devices that belong
# to that particular phone brand
phone_brand_counts = brand_df.phone_brand_id.value_counts(normalize=True).rename_axis(
    'phone_brand_id').reset_index(name='proportion').merge(brand_categories,
                                                       on='phone_brand_id')
phone_brand_counts.head(5) # top 10 phone brands

,phone_brand_id,proportion,phone_brand
0,0,0.230767,小米
1,1,0.183108,三星
2,3,0.173911,华为
3,4,0.076878,vivo
4,8,0.076312,OPPO


### App Labels: Put into subcategories

In [40]:
import spacy
# Load the spacy model that you have installed - for pre-trained word embeddings
# if NOT downloaded, run command: python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_md')

In [77]:
# determine important label_categories
app_category_ls = list(label_categories.category)
# possibly subcategorize into games, comic, shopping, education, banking/finance, travel
print('There are ', len(app_category_ls), 'app labels in label_categories dataframe')

# get all unique values in list of app labels: 
category_ls = list(set(app_category_ls))
print('There are ', len(category_ls), 'unique app labels')

There are  930 app labels in label_categories dataframe
There are  836 unique app labels


In [78]:
# Preprocessing of category list: 

# remove nan value and convert to df:
category_df = pd.DataFrame([word for word in category_ls if type(word) == str], columns =['labels'])

# remove all labels that have to do with games: 
category_ls = list(category_df[~category_df['labels'].str.contains("game|poker|Game|majiang", na=False)].labels)
category_ls.append('game') # convert back to list and append string with game to encompass all games

In [108]:
# create new column for revised versions of the labels
category_df['revised_labels'] = category_df.labels

# put the revised label names in a dataframe with original label names (e.g. all those involving game are just game)
category_df.loc[category_df['revised_labels'].str.contains('game|poker|Game|majiang|chess', 
                                                           case=False), 'revised_labels'] = 'game'

# fix all the mispelled/not recognized labels
category_df.loc[category_df.labels == 'communitation','revised_labels'] = 'communication'
category_df.loc[category_df.labels == 'mesasge','revised_labels'] = 'message'

category_df.loc[category_df.labels.str.contains('operatior'),'revised_labels'] = 'operator'
category_df.loc[category_df.labels.str.contains('managemetn'),'revised_labels'] = 'class management'
category_df.loc[category_df.labels.str.contains(r'paid\d+$'),'revised_labels'] = 'paid'

In [128]:
# predefined categories
category_subsets = ['shopping','travel','entertainment','food','comic','education',
                    'media','finance','family','technology','other']

# go through and assign each label a category using nlp

category_assigns = []
for i,category in enumerate(list(category_df.revised_labels)):
    similarity = dict(zip(category_subsets, [0]*len(category_subsets)))
    tokens = nlp(category)
    for word in tokens:
        if(word and word.vector_norm):
            for subset in category_subsets:
                similarity[subset] += word.similarity(nlp(subset))
        else: 
            print(word, tokens)
            
    if all(elem==0 for elem in list(similarity.values())):
        category_assigns.append((category, 'other'))
    else:
        max_value= max(similarity, key=lambda key: similarity[key])
        category_assigns.append((category, max_value))
    


0it [00:00, ?it/s]
1it [00:00,  2.12it/s]
2it [00:00,  2.67it/s]
3it [00:00,  3.26it/s]
4it [00:01,  3.47it/s]
5it [00:01,  4.13it/s]
6it [00:01,  4.56it/s]
7it [00:02,  2.48it/s]
8it [00:02,  3.11it/s]
9it [00:02,  3.01it/s]
10it [00:03,  2.61it/s]
11it [00:03,  2.57it/s]
12it [00:03,  3.19it/s]
13it [00:04,  2.80it/s]
14it [00:04,  2.51it/s]
15it [00:04,  3.13it/s]
16it [00:04,  3.79it/s]
17it [00:05,  3.88it/s]
18it [00:05,  3.12it/s]
19it [00:05,  3.31it/s]
20it [00:06,  3.46it/s]
21it [00:06,  3.54it/s]
22it [00:06,  4.10it/s]
23it [00:06,  4.69it/s]
24it [00:06,  5.26it/s]

DS_P2P DS_P2P net loan



25it [00:07,  4.82it/s]
26it [00:07,  3.25it/s]
27it [00:07,  3.93it/s]
28it [00:07,  4.53it/s]
29it [00:08,  3.41it/s]
30it [00:08,  3.56it/s]
31it [00:08,  4.22it/s]
32it [00:08,  4.88it/s]
33it [00:09,  3.91it/s]
34it [00:09,  3.50it/s]
35it [00:09,  3.53it/s]
36it [00:10,  3.60it/s]
37it [00:10,  3.73it/s]
38it [00:10,  4.36it/s]
39it [00:10,  4.90it/s]
40it [00:10,  5.34it/s]
41it [00:10,  5.87it/s]
42it [00:11,  6.23it/s]
43it [00:11,  6.35it/s]
44it [00:11,  4.47it/s]
45it [00:11,  4.32it/s]
46it [00:12,  4.83it/s]
47it [00:12,  5.40it/s]
48it [00:12,  3.18it/s]
49it [00:13,  3.27it/s]
50it [00:13,  3.82it/s]
51it [00:13,  4.41it/s]
52it [00:13,  4.94it/s]
53it [00:13,  4.65it/s]
54it [00:13,  5.04it/s]
55it [00:14,  5.40it/s]
56it [00:14,  5.91it/s]
57it [00:14,  6.38it/s]
58it [00:14,  5.43it/s]
59it [00:14,  5.90it/s]
60it [00:15,  4.43it/s]
61it [00:15,  4.29it/s]
62it [00:15,  4.85it/s]
63it [00:15,  4.61it/s]
64it [00:16,  3.75it/s]
65it [00:16,  3.84it/s]
66it [00:16,  3

dotal dotal-lol



417it [01:41,  4.92it/s]
418it [01:41,  4.55it/s]
419it [01:42,  5.18it/s]
420it [01:42,  5.70it/s]
421it [01:42,  6.18it/s]
422it [01:42,  6.46it/s]
423it [01:42,  4.69it/s]
424it [01:42,  5.27it/s]
425it [01:43,  4.83it/s]
426it [01:43,  4.60it/s]
427it [01:43,  5.24it/s]
428it [01:43,  4.86it/s]
429it [01:44,  4.60it/s]
430it [01:44,  4.44it/s]
431it [01:45,  2.15it/s]
432it [01:45,  2.52it/s]
433it [01:45,  2.84it/s]
434it [01:45,  3.48it/s]
435it [01:46,  2.67it/s]
436it [01:46,  2.98it/s]
437it [01:47,  3.16it/s]
438it [01:47,  3.25it/s]
439it [01:47,  3.30it/s]
440it [01:47,  3.33it/s]
441it [01:48,  3.40it/s]
442it [01:48,  3.95it/s]
443it [01:48,  4.47it/s]
444it [01:48,  4.81it/s]
445it [01:48,  5.24it/s]
446it [01:49,  4.59it/s]
447it [01:49,  4.26it/s]
448it [01:49,  4.03it/s]
449it [01:49,  4.57it/s]
450it [01:50,  3.33it/s]
451it [01:50,  3.51it/s]
452it [01:50,  4.16it/s]
453it [01:50,  4.83it/s]
454it [01:50,  5.37it/s]
455it [01:51,  4.91it/s]
456it [01:51,  5.55it/s]

...


467it [01:53,  5.41it/s]
468it [01:53,  5.86it/s]
469it [01:53,  6.37it/s]
470it [01:54,  4.54it/s]
471it [01:54,  5.14it/s]
472it [01:54,  5.69it/s]
473it [01:54,  6.13it/s]
474it [01:54,  5.27it/s]
475it [01:54,  5.86it/s]
476it [01:55,  6.37it/s]
477it [01:55,  6.68it/s]
478it [01:55,  6.91it/s]
479it [01:55,  5.76it/s]
480it [01:55,  5.05it/s]
481it [01:56,  4.73it/s]
482it [01:56,  4.51it/s]
483it [01:56,  3.41it/s]
484it [01:57,  3.60it/s]
485it [01:57,  3.75it/s]
486it [01:57,  4.44it/s]
487it [01:57,  4.36it/s]
488it [01:57,  5.03it/s]
489it [01:57,  5.51it/s]
490it [01:58,  5.03it/s]
491it [01:58,  5.63it/s]
492it [01:58,  6.18it/s]
493it [01:58,  3.85it/s]
494it [01:59,  3.94it/s]
495it [01:59,  3.57it/s]
496it [01:59,  4.26it/s]
497it [01:59,  4.88it/s]
498it [01:59,  4.59it/s]
499it [02:00,  5.27it/s]
500it [02:00,  4.82it/s]
501it [02:00,  3.19it/s]
502it [02:01,  3.87it/s]
503it [02:01,  3.98it/s]
504it [02:01,  4.01it/s]
505it [02:01,  4.65it/s]
506it [02:01,  5.27it/s]

QDII QDII fund



652it [02:37,  4.34it/s]
653it [02:37,  4.27it/s]
654it [02:37,  4.24it/s]
655it [02:37,  3.30it/s]
656it [02:38,  3.55it/s]
657it [02:38,  3.28it/s]
658it [02:38,  3.47it/s]
659it [02:39,  3.65it/s]
660it [02:39,  3.80it/s]
661it [02:39,  4.49it/s]
662it [02:39,  4.30it/s]
663it [02:39,  4.88it/s]
664it [02:40,  4.59it/s]
665it [02:40,  5.26it/s]
666it [02:40,  4.80it/s]
667it [02:40,  4.52it/s]
668it [02:40,  5.12it/s]
669it [02:41,  4.84it/s]
670it [02:41,  5.45it/s]
671it [02:41,  4.98it/s]
672it [02:41,  5.51it/s]
673it [02:41,  4.99it/s]
674it [02:42,  4.69it/s]
675it [02:42,  4.46it/s]
676it [02:42,  4.37it/s]
677it [02:42,  4.10it/s]
678it [02:42,  4.76it/s]
679it [02:43,  5.44it/s]
680it [02:43,  6.03it/s]
681it [02:43,  5.30it/s]
682it [02:43,  4.19it/s]
683it [02:44,  4.16it/s]
684it [02:44,  4.80it/s]
685it [02:44,  5.40it/s]
686it [02:44,  4.92it/s]
687it [02:44,  4.64it/s]
688it [02:44,  5.18it/s]
689it [02:45,  4.91it/s]
690it [02:45,  4.61it/s]
691it [02:45,  5.20it/s]

Shushan Shushan



807it [03:10,  6.24it/s]
808it [03:11,  4.56it/s]
809it [03:11,  5.21it/s]
810it [03:11,  5.71it/s]
811it [03:11,  6.20it/s]
812it [03:11,  5.29it/s]
813it [03:12,  4.12it/s]
814it [03:12,  3.21it/s]
815it [03:12,  3.42it/s]
816it [03:12,  4.13it/s]
817it [03:13,  4.82it/s]
818it [03:13,  5.46it/s]
819it [03:13,  3.04it/s]
820it [03:14,  3.30it/s]
821it [03:14,  3.95it/s]
822it [03:14,  4.59it/s]
823it [03:14,  5.21it/s]
824it [03:14,  4.74it/s]
825it [03:14,  4.50it/s]
826it [03:15,  4.41it/s]
827it [03:15,  5.07it/s]
828it [03:15,  4.76it/s]
829it [03:15,  5.33it/s]
830it [03:15,  5.87it/s]
831it [03:15,  6.29it/s]
832it [03:16,  6.66it/s]
833it [03:16,  6.95it/s]
834it [03:16,  5.79it/s]
835it [03:16,  4.24it/s]


In [142]:
# save resulting assignments and assign to new column in category_df
res = [[ i for i, j in category_assigns ], 
       [ j for i, j in category_assigns ]] 
category_df['general_category'] = res[1]

In [145]:
category_df.head()

,labels,revised_labels,general_category
0,Maternal and child supplies,Maternal and child supplies,family
1,Outlaws of the Marsh game,game,entertainment
2,Living,Living,family
3,convenience services,convenience services,shopping
4,game-Racing,game,entertainment


In [149]:
# merge the new information into the original label_categories dataframe
label_categories.rename(columns = {'category': 'labels'}, inplace=True)
label_categories = label_categories.merge(category_df[['labels','general_category']], on='labels', how='outer')
label_categories.fillna(value={'general_category': 'other'}, inplace=True)
label_categories[label_categories.labels.isna()]

In [163]:
# merge the new information into a new dataframe with app_ids 
app_categories = app_labels.merge(label_categories[['label_id','general_category']], on='label_id')

In [170]:
app_categories.head()

,app_id,label_id,general_category
0,7324884708820027918,251,finance
1,-4494216993218550286,251,finance
2,8756705988821000489,251,finance
3,1061207043315821111,251,finance
4,-1491198667294647703,251,finance


# Creating a reference table that gathers the informations in the multiples tables

Merge the tables into one general table so we can access the data more easily

In [13]:
complete_ref_table = app_events
complete_ref_table = complete_ref_table.merge(app_labels, on = "app_id")
complete_ref_table = complete_ref_table.merge(label_categories, on = "label_id")
del complete_ref_table['is_installed']  # I dont think is_installed adds any information
complete_ref_table = complete_ref_table.merge(events, on="event_id")
complete_ref_table = complete_ref_table.merge(brand_df, on="device_id")
complete_ref_table = complete_ref_table[["event_id", "app_id", "label_id", "category", "device_id", "phone_brand", "device_model", "timestamp", "longitude", "latitude", "is_active"]]

display(complete_ref_table.head())

,event_id,app_id,label_id,category,device_id,phone_brand,device_model,timestamp,longitude,latitude,is_active
0,2,5927333115845830913,549,Property Industry 1.0,-6401643145415154744,三星,Galaxy Grand Prime,2016-05-01 00:54:12,103.65,30.97,1
1,2,8693964245073640147,549,Property Industry 1.0,-6401643145415154744,三星,Galaxy Grand Prime,2016-05-01 00:54:12,103.65,30.97,1
2,2,4775896950989639373,549,Property Industry 1.0,-6401643145415154744,三星,Galaxy Grand Prime,2016-05-01 00:54:12,103.65,30.97,1
3,2,-8022267440849930066,549,Property Industry 1.0,-6401643145415154744,三星,Galaxy Grand Prime,2016-05-01 00:54:12,103.65,30.97,0
4,2,9112463267739110219,549,Property Industry 1.0,-6401643145415154744,三星,Galaxy Grand Prime,2016-05-01 00:54:12,103.65,30.97,0


# Creating training set

Once we have this reference table we can build our training set, assigning to every entry of that table the corresponding age and gender for the user

In [14]:
# Build training set
train_set = (complete_ref_table.merge(gender_age_train, on="device_id")).head(3500000)
test_set = complete_ref_table.merge(gender_age_train, on="device_id").tail(200000)
test_set.head()

,event_id,app_id,label_id,category,device_id,phone_brand,device_model,timestamp,longitude,latitude,is_active,gender,age,group
2294132,69857,33792862810792679,252,Wealth Management,-8649941211508968787,小米,红米,2016-05-04 00:20:57,0.0,0.0,1,M,62,M39+
2294133,69857,3683147815759994238,252,Wealth Management,-8649941211508968787,小米,红米,2016-05-04 00:20:57,0.0,0.0,1,M,62,M39+
2294134,69857,6965654211116534216,252,Wealth Management,-8649941211508968787,小米,红米,2016-05-04 00:20:57,0.0,0.0,0,M,62,M39+
2294135,69857,6965654211116534216,761,Financial Services,-8649941211508968787,小米,红米,2016-05-04 00:20:57,0.0,0.0,0,M,62,M39+
2294136,69857,502665604573257504,761,Financial Services,-8649941211508968787,小米,红米,2016-05-04 00:20:57,0.0,0.0,1,M,62,M39+
